In [1]:
import pandas
import datetime
import geomag
import numpy

In [2]:
# some constants
cutOffLosVel = 50.

In [3]:
#### In this block we load Velocity data ####
#### In this block we load Velocity data ####
#### In this block we load Velocity data ####
# need the formatted velocity file to read data
outFormattedFile = "../data/new-test-vels-north.txt"
# os.system( 'cat ' + rawVelFileName + ' | tr -s "  " > ' + outFormattedFile )
inpColNames = [ "dateStr", "timeStr", "beam", "range", \
          "geoAzm", "azimCalcMag", "vLos", "spwdth", "pwr", "MLAT", "MLON", "MLT",  \
          "GLAT", "GLON", "radId", "radCode"]
velsDataDF = pandas.read_csv(outFormattedFile, delim_whitespace=True,\
                             header=None, names=inpColNames)
# add a datetime col
velsDataDF["date"] = pandas.to_datetime( \
                        velsDataDF['dateStr'].astype(str) + "-" +\
                        velsDataDF['timeStr'].astype(str), format='%Y%m%d-%H%M')
# for some reason MLAT is a str type, convert it to float
velsDataDF["MLAT"] = velsDataDF["MLAT"].astype(float)
# Also get a normMLT for plotting
velsDataDF['normMLT'] = [x-24 if x >= 12 else x for x in velsDataDF['MLT']]
velsDataDF.head()
#### In this block we load SAPS Velocity data ####
#### In this block we load SAPS Velocity data ####
#### In this block we load SAPS Velocity data ####

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT
0,20110409,600,0,26,-60.95,-73.7007,-2.2716,1.8018,16.9756,53.2044,-21.8611,23.6993,41.7498,-91.5990,33,bks,2011-04-09 06:00:00,-0.3007
1,20110409,600,0,27,-60.95,-74.3564,-5.1056,1.8336,19.5614,53.3065,-22.5771,23.6515,41.8966,-92.1093,33,bks,2011-04-09 06:00:00,-0.3485
2,20110409,600,0,28,-60.95,-75.0118,-5.1056,6.4063,24.4064,53.4035,-23.2954,23.6036,42.0404,-92.6216,33,bks,2011-04-09 06:00:00,-0.3964
3,20110409,600,0,29,-60.95,-75.6670,-4.5666,6.4063,19.5614,53.4954,-24.0159,23.5556,42.1813,-93.1360,33,bks,2011-04-09 06:00:00,-0.4444
4,20110409,600,0,30,-60.95,-76.3220,-3.6944,0.0100,17.1926,53.5823,-24.7384,23.5074,42.3191,-93.6523,33,bks,2011-04-09 06:00:00,-0.4926


In [4]:
# convert azimuth to magn coords
gm = geomag.geomag.GeoMag()
def convert_to_mag_azm(row):
#     print type(row["date"].to_datetime()),row["date"].to_datetime()
    mg = gm.GeoMag(row['GLAT'], \
                   row['GLON'], h=300., time=row['date'].date())
    azm_mag = (round(row['geoAzm'] - mg.dec,2))
    return azm_mag


velsDataDF["magAzm"] = -999.#velsDataDF.apply( convert_to_mag_azm, axis=1 )
velsDataDF.head()

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,magAzm
0,20110409,600,0,26,-60.95,-73.7007,-2.2716,1.8018,16.9756,53.2044,-21.8611,23.6993,41.7498,-91.5990,33,bks,2011-04-09 06:00:00,-0.3007,-999.0
1,20110409,600,0,27,-60.95,-74.3564,-5.1056,1.8336,19.5614,53.3065,-22.5771,23.6515,41.8966,-92.1093,33,bks,2011-04-09 06:00:00,-0.3485,-999.0
2,20110409,600,0,28,-60.95,-75.0118,-5.1056,6.4063,24.4064,53.4035,-23.2954,23.6036,42.0404,-92.6216,33,bks,2011-04-09 06:00:00,-0.3964,-999.0
3,20110409,600,0,29,-60.95,-75.6670,-4.5666,6.4063,19.5614,53.4954,-24.0159,23.5556,42.1813,-93.1360,33,bks,2011-04-09 06:00:00,-0.4444,-999.0
4,20110409,600,0,30,-60.95,-76.3220,-3.6944,0.0100,17.1926,53.5823,-24.7384,23.5074,42.3191,-93.6523,33,bks,2011-04-09 06:00:00,-0.4926,-999.0


In [5]:
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
# remove velocies whose magnitude is less than 200 m/s
velsDataDF = velsDataDF[ abs(velsDataDF["vLos"]) >= cutOffLosVel ]
# SAPS(westward) vLos are positive for positive azimuths and vice versa.
# filter the others out
velsDataDF = velsDataDF[ velsDataDF["azimCalcMag"]/velsDataDF["vLos"] > 0. ].reset_index(drop=True)
velsDataDF.head()
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,magAzm
0,20110409,632,9,62,-26.21,-56.2581,-70.9955,62.3067,8.6932,68.5471,-36.3932,23.2623,58.8567,-100.0848,33,bks,2011-04-09 06:32:00,-0.7377,-999.0
1,20110409,632,9,63,-26.21,-57.1024,-60.7902,86.8048,10.5512,68.7611,-37.2961,23.2021,59.1557,-100.6260,33,bks,2011-04-09 06:32:00,-0.7979,-999.0
2,20110409,632,10,61,-22.35,-48.4472,-89.6343,62.1669,8.1359,69.5836,-31.9161,23.5607,59.6395,-97.0140,33,bks,2011-04-09 06:32:00,-0.4393,-999.0
3,20110409,632,10,62,-22.35,-49.2396,-74.9928,69.3493,11.5324,69.8388,-32.7600,23.5045,59.9672,-97.4991,33,bks,2011-04-09 06:32:00,-0.4955,-999.0
4,20110409,632,10,63,-22.35,-50.0515,-60.7902,92.1583,12.0293,70.0900,-33.6236,23.4469,60.2934,-97.9938,33,bks,2011-04-09 06:32:00,-0.5531,-999.0


In [6]:
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
datFileName = "../data/processedSaps.txt"
sapsDataDF = pandas.read_csv(datFileName, sep=' ', dtype={'dateStr':'str', 'time': 'str'})

sapsDataDF["date"] = pandas.to_datetime( \
                        sapsDataDF['dateStr'] + "-" +\
                        sapsDataDF['time'], format='%Y%m%d-%H%M')
sapsDataDF.head()
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####

,dateStr,sapsLat,sapsMLT,sapsVel,radId,poesLat,poesMLT,dst_date,dst_index,time,date
0,20110107,56.5,17.7543,308.2077,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
1,20110107,55.5,18.0147,224.1588,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
2,20110107,56.5,17.8749,307.4328,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
3,20110107,55.5,18.1324,222.4787,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
4,20110107,56.5,17.9955,305.4201,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07


In [7]:

#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
# we choose 0840 UT since that time was analyzed in the paper
timeSlctd = datetime.datetime( 2011, 4, 9, 7, 0 )
velAnlysDF = velsDataDF[ velsDataDF["date"] == timeSlctd ].reset_index(drop=True)
sapsSelPrdDF = sapsDataDF[  ( sapsDataDF["date"] - timeSlctd < numpy.timedelta64(40,'m') )\
           & ( sapsDataDF["date"] - timeSlctd > numpy.timedelta64(0,'m') )  ].reset_index(drop=True)
# Now check if there is more than one time period in the selected interval
# ideally there shouldn't be, but if we find one we should do something
# about it (like take an average)...for now its undecided.!
if len( sapsSelPrdDF["time"].unique() ) > 1:
    print "There is more than one time period selected!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
poesBndDF = sapsSelPrdDF[ ["poesMLT", "poesLat"] \
                    ].drop_duplicates().reset_index(drop=True)
# Have a normalized MLT for ease of comparison
poesBndDF['normMLT'] = [x-24 if x >= 12 else x for x in poesBndDF['poesMLT']]
poesBndDF.head()
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####

,poesMLT,poesLat,normMLT
0,0.0,62.2512,0.0
1,1.0,61.5701,1.0
2,2.0,61.1526,2.0
3,22.0,64.2041,-2.0
4,23.0,63.1496,-1.0


In [8]:
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
# Merge POES boundary DF with the vels DF
velAnlysDF["normMLTRound"] = velAnlysDF["normMLT"].astype(int)
print "shape before join--->", velAnlysDF.shape
velAnlysDF = pandas.merge( velAnlysDF, poesBndDF, left_on="normMLTRound", right_on="normMLT", how="inner" )
print "shape after join--->", velAnlysDF.shape
# Filter out velocties above the POES boundary
velAnlysDF = velAnlysDF[ velAnlysDF["MLAT"] < velAnlysDF["poesLat"]\
                       ].reset_index(drop=True).drop_duplicates()
print "shape after filtering by boundary--->", velAnlysDF.shape
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####

shape before join---> (431, 20)
shape after join---> (397, 23)
shape after filtering by boundary---> (302, 23)


In [9]:
# save the data in a csv for lshell fitting
outSelCols = [ "beam", "range", "geoAzm", "azimCalcMag", "magAzm", "vLos"\
           , "MLAT", "MLT", "MLON", "GLAT", "GLON", "radId"\
           , "radCode","normMLT_x", "normMLTRound", "spwdth", "pwr" ]
velAnlysDF.to_csv( "../data/test-losVels.txt", columns=outSelCols, index=False )